# Importaciones

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
import pandas as pd
import numpy as np
import os

<ipython-input-3-8d2f0729e833>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


# Definición del modelo

In [ ]:
# Definir el modelo de la CNN
def build_cnn(hp):
    model = Sequential([
        Conv2D(hp.Int('filters_1', 32, 64, step=16), (3,3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2,2),
        Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1)),

        Conv2D(hp.Int('filters_2', 64, 128, step=16), (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1)),

        Conv2D(hp.Int('filters_3', 128, 256, step=32), (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(hp.Float('dropout_3', 0.3, 0.6, step=0.1)),

        Flatten(),
        Dense(hp.Int('dense_units', 64, 256, step=64), activation='relu'),
        Dropout(hp.Float('dropout_dense', 0.3, 0.6, step=0.1)),
        Dense(3, activation='softmax')
    ])

    optimizer = Adam(learning_rate=hp.Choice('learning_rate', [1e-4, 5e-4, 1e-3]))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', 'precision', 'recall', 'AUC'])

    return model

# Carga de datos

In [ ]:
# Definir generadores de datos
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Directorios de imágenes
train_dir = '/content/drive/MyDrive/covid-chest-xray'
test_dir = '/content/drive/MyDrive/covid-chest-xray'

# Obtener número de imágenes por clase en el conjunto de entrenamiento
train_class_counts = {cls: len(os.listdir(os.path.join(train_dir, cls))) for cls in os.listdir(train_dir)}
min_class_count = min(train_class_counts.values())  # Tomar la cantidad mínima para validación

# Generadores de imágenes
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True)

# Generador de test con datos originales
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


# Entrenamiento del modelo

In [ ]:
# Configurar Random Search
tuner = RandomSearch(
    build_cnn,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=1,
    directory='cnn_chest-xray_problem2',
    project_name='cnn_chest-xray_problem2'
)

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Realizar la búsqueda de hiperparámetros
tuner.search(train_generator,
             validation_data=test_generator,
             epochs=20,
             callbacks=[early_stopping],
             verbose=1)

In [ ]:
tuner.results_summary()

In [ ]:
# Obtener el mejor modelo
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]
print(f"Mejor modelo encontrado con filtros: {best_hps.get('filters_1')}, {best_hps.get('filters_2')}, {best_hps.get('filters_3')}, learning rate: {best_hps.get('learning_rate')}")

Mejor modelo encontrado con filtros: 48, 96, 256, learning rate: 0.0001


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Extraer el historial de entrenamiento del mejor modelo
history = best_model.fit(train_generator, validation_data=test_generator, epochs=20)

# Convertir en DataFrame para análisis posterior
df_metrics = pd.DataFrame(history.history)

# Guardar como CSV para análisis posterior
df_metrics.to_csv("training_metrics.csv", index=False)
print("Métricas guardadas en 'training_metrics.csv'")

In [ ]:
# Guardar el mejor modelo
best_model.save('best_cnn_model.h5')
print("Mejor modelo guardado como 'best_cnn_model.h5'")

Mejor modelo guardado como 'best_cnn_model.h5'
